In [ ]:
# stdlib
import asyncio
import logging
from pathlib import Path
import sys

# third party
import veilid

project_root = Path.cwd().parent.parent.parent
veilid_path = project_root / "packages" / "grid" / "veilid" / "server"
sys.path.append(veilid_path.as_posix())

# third party
from veilid_streamer import VeilidStreamer  # type: ignore

logging.getLogger('veilid_streamer').setLevel(logging.DEBUG)

In [ ]:
host = "localhost"
port = 5959

In [ ]:
async def veilid_callback(
    update: veilid.VeilidUpdate, app_message_queue: asyncio.Queue
):
    if update.kind in {
        veilid.VeilidUpdateKind.APP_MESSAGE,
        veilid.VeilidUpdateKind.APP_CALL,
    }:
        await app_message_queue.put(update)


app_message_queue = asyncio.Queue()
conn = await veilid.json_api_connect(
    host, port, lambda update: veilid_callback(update, app_message_queue)
)

In [ ]:
router = await (await conn.new_routing_context()).with_default_safety()
record = await router.create_dht_record(veilid.DHTSchema.dflt(1))
public_key, private_key = record.owner, record.owner_secret
await router.close_dht_record(record.key)
key_pair = veilid.KeyPair.from_parts(key=public_key, secret=private_key)
record_open = await router.open_dht_record(record.key, key_pair)
route_id, blob = await conn.new_custom_private_route(
    [veilid.CryptoKind.CRYPTO_KIND_VLD0],
    veilid.Stability.RELIABLE,
    veilid.Sequencing.ENSURE_ORDERED,
)
await router.set_dht_value(record_open.key, 0, blob)
self_remote_private_route = await conn.import_remote_private_route(blob)
await router.app_message(self_remote_private_route, b"READY")
update = await app_message_queue.get()
assert update.detail.message == b"READY"
print(f"Your DHT Key: {record.key}")

In [ ]:
vs = VeilidStreamer(connection=conn, router=router)


async def receive_stream_callback(message: bytes) -> bytes:
    response = f"Received {len(message)} bytes."
    print(response)
    return response.encode()

# Reset the queue
app_message_queue = asyncio.Queue()

while True:
    update = await app_message_queue.get()
    if vs.is_stream_update(update):
        await vs.receive_stream(update, callback=receive_stream_callback)
    elif update.detail.message == b"QUIT":
        print("Received QUIT message. Exiting.")
        break
    else:
        print(f"Received: {update.detail.message}")